In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model_name = "mistralai/Mistral-Small-24B-Instruct-2501"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=quant_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-05-02 07:21:58.960976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746170519.456890   21679 cuda_dnn.cc:8579] Unable to register cuD

Loading QLoRA checkpoint and adding adapters for the base model

In [2]:
from peft import PeftModel
merged_model = PeftModel.from_pretrained(model, "./mistral-24b-clm-qlora/checkpoint-1796")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
system_prompt = "Ты — полезный и вежливый помощник ассистент-диетолог, который отвечает кратко и по делу."
user_prompt = "как приготовить фарш для макарон по-флотски?"

full_prompt = f"<s>[SYSTEM] {system_prompt} [/SYSTEM]\n[USER] {user_prompt} [/USER]\n[ASSISTANT]"
input_ids = tokenizer.encode(full_prompt, return_tensors="pt").to(device)

In [22]:
with torch.no_grad():
    output = merged_model.generate(
        input_ids,
        max_length=512,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
match = re.search(r"\[ASSISTANT\](.*?)\[/ASSISTANT\]", generated_text, re.DOTALL)
if match:
    assistant_text = match.group(1).strip()

answer = f'Пользователь: {user_prompt} \nАссистент: {assistant_text}'
print(answer)

Пользователь: как приготовить фарш для макарон по-флотски? 
Ассистент: Для приготовления фарша для макарон по-флотски вам потребуется:
- фарш (говяжий, свиной или смешанный) — 400 г
- лук — 1 шт
- морковь — 1 шт
- томатная паста — 2 ст. л. - соль, перец, специи (паприка, чеснок) — по вкусу
- растительное масло — для жарки
- макароны — 400 г
- вода — для варки макарон
- сливочное масло — 1 ст. л. - петрушка — для украшения
Приготовление:
1. Мелко нарежьте лук и морковь. Обжарьте их на растительном масле до золотистого цвета. 2. Добавьте фарш, обжарьте до готовности, постоянно помешивая. 3. Добавьте томатную пасту, соль, перец и специи. Тушите фарш еще 5 минут. 4. Отварите макароны в подсоленной воде до готовности. 5. Смешайте макароны с фаршем, добавьте сливочное масло. 6. Подавайте, украсив петрушкой. Приятного аппетита! 
При необходимости, можно добавить немного воды из-под макарон, чтобы соус был более жидким.
